In [1]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations
AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]

In [2]:
dipep = [''.join(i) for i in itertools.product(AA, repeat = 2)]

In [3]:
arr = numpy.empty((0,402), dtype=numpy.float)
names = numpy.empty((0,1),  dtype=object)
for record in SeqIO.parse('A45_phage_orfs.txt', "fasta"):
        ll=len(record.seq)
        #print(record.id)
        #print(ll)
        #print(record.seq)
        X = ProteinAnalysis(record.seq.__str__().replace('X','A').replace('J','L'))
        tt= [X.isoelectric_point(), X.instability_index()]
        tt_n = numpy.asarray(tt,dtype=numpy.float)

        dipep_count=[record.seq.count(i)/ll for i in dipep]
        dipep_count_n = numpy.asarray(dipep_count,dtype=numpy.float)
    
        cat_n= numpy.append(dipep_count_n,tt_n)
        cat_n = cat_n.reshape((1,cat_n.shape[0]))

        arr = numpy.append(arr,cat_n , axis=0)
        names = numpy.append(names,record.id)
        

In [4]:
numpy.set_printoptions(formatter={'float': '{: 0.3f}'.format})
print(arr)


[[ 0.012  0.002  0.009 ...  0.000  6.208  40.227]
 [ 0.008  0.003  0.007 ...  0.000  5.895  43.471]
 [ 0.019  0.000  0.010 ...  0.000  5.218  37.563]
 ...
 [ 0.000  0.003  0.000 ...  0.000  8.785  39.862]
 [ 0.009  0.000  0.000 ...  0.000  6.001  57.585]
 [ 0.005  0.000  0.005 ...  0.005  5.952  31.812]]


In [5]:
import pickle
mean_arr=pickle.load(open( "dipep_new_mean.p", "rb" ) )
std_arr=pickle.load(open( "dipep_new_std.p", "rb" ) )

In [6]:
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        arr[i,j]=(arr[i,j]-mean_arr[j])/std_arr[j]

In [7]:
print(arr)

[[ 0.402  1.406  0.808 ... -0.540  0.073  0.619]
 [-0.056  1.631  0.338 ... -0.540 -0.118  0.944]
 [ 1.318 -0.323  0.859 ... -0.540 -0.531  0.352]
 ...
 [-1.022  2.091 -1.096 ... -0.540  1.644  0.583]
 [ 0.094 -0.323 -1.096 ... -0.540 -0.054  2.356]
 [-0.398 -0.323 -0.053 ...  1.905 -0.083 -0.223]]


In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.models import load_model

model = load_model('di_new_model.h5')

Using TensorFlow backend.


In [9]:
print(model.predict_classes(arr))

[2 5 0 5 9 0 2 4 6 7 2 8 2 4 4 3 3 2 5 3 2 1 6 2 5 8 0 0 6 0 3 1 5 6 9 0 0
 5 4 5 2 6 2 0 9 5 2 2 5]


In [10]:
print(model.predict(arr)[...,0])


[ 0.000  0.000  0.731  0.000  0.000  1.000  0.001  0.011  0.160  0.030
  0.000  0.000  0.002  0.280  0.001  0.000  0.001  0.000  0.203  0.000
  0.000  0.000  0.000  0.000  0.000  0.000  1.000  1.000  0.051  1.000
  0.000  0.000  0.000  0.000  0.013  0.793  0.995  0.000  0.000  0.000
  0.007  0.000  0.000  0.922  0.000  0.022  0.182  0.000  0.000]


In [11]:
arr_pred=model.predict(arr)

In [15]:
col_names=["major capsid","minor capsid","baseplate",
           "major tail","minor tail","portal",
           "tail fiber","tail shaft","colar",
           "HTJ"]

In [16]:
import pandas as pd
kk=pd.DataFrame(data=arr_pred,
                   index=names,
                    columns=col_names
                   )

display(kk)

,major capsid,minor capsid,baseplate,major tail,minor tail,portal,tail fiber,tail shaft,colar,HTJ
1_A45,0.000,0.000,0.990,0.000,0.000,0.010,0.000,0.000,0.000,0.000
2_A45,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000
3_A45,0.731,0.000,0.029,0.000,0.000,0.235,0.002,0.000,0.000,0.002
4_A45,0.000,0.000,0.042,0.000,0.000,0.958,0.000,0.000,0.000,0.000
5_A45,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.999
6_A45,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
7_A45,0.001,0.000,0.710,0.002,0.158,0.009,0.000,0.000,0.000,0.120
8_A45,0.011,0.000,0.023,0.008,0.906,0.007,0.001,0.035,0.000,0.007
9_A45,0.160,0.049,0.011,0.001,0.027,0.000,0.556,0.006,0.025,0.165
10_A45,0.030,0.023,0.287,0.002,0.170,0.014,0.011,0.422,0.036,0.007


In [22]:
pd.options.display.float_format = '{:,.3f}'.format
#kk.style.apply(highlight_max,axis=1)
html=kk.style.format("{:.2f}").highlight_max(axis=1).render()

In [23]:
file = open('A45_form.html', 'w')
file.write(html)
file.close()